# kaggle_quora: 5-fold ensemble of yuhaitao

比赛baseline

参考:

https://www.kaggle.com/shujian/single-rnn-with-4-folds-clr

https://www.kaggle.com/gmhost/gru-capsule

https://github.com/dennybritz/cnn-text-classification-tf

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test.csv', 'train.csv', 'embeddings', 'embeddings.zip']


# load package

In [2]:
import os
import time
import random
import re
from tqdm import tqdm
from IPython.display import display
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from collections import Counter

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


# global parameters

In [3]:
data_dir = "../input/"
train_file = os.path.join(data_dir, "train.csv")
test_file = os.path.join(data_dir, "test.csv")
embedding_size = 300
max_len = 50
max_features = 120000
batch_size = 512
use_local_test = True

# Data preprocess

In [4]:
# 将特殊字符单独挑出
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
#             x = x.replace(punct, f' {punct} ') # 这是python3.6语法
            x = x.replace(punct, ' '+punct+' ')
    return x

# 清洗数字
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

# 清洗拼写
mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [5]:
def load_and_prec(use_local_test=True):
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    display(train_df.head())
    display(test_df.head())
    
    # 小写
    train_df["question_text"] = train_df["question_text"].str.lower()
    test_df["question_text"] = test_df["question_text"].str.lower()
    
    # 数字清洗
    train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_numbers(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_numbers(x))
    
    # 清洗拼写
    train_df["question_text"] = train_df["question_text"].apply(lambda x: replace_typical_misspell(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: replace_typical_misspell(x))
    
    # 数据清洗
    train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))
    
    ## fill up the missing values
    train_X = train_df["question_text"].fillna("_##_").values
    test_X = test_df["question_text"].fillna("_##_").values

    ## Tokenize the sentences
    # 这个方法把所有字母都小写了
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    test_X = tokenizer.texts_to_sequences(test_X)
    
    ## Get the target values
    train_Y = train_df['target'].values
    print(np.sum(train_Y))
    
#     # 在pad之前把前30个词去掉
#     train_cut = []
#     test_cut = []
#     for x in train_X:
#         train_cut.append([i for i in x if i>30])
#     for x in test_X:
#         test_cut.append([i for i in x if i>30])
#     train_X = train_cut
#     test_X = test_cut

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=max_len, padding="post", truncating="post")
    test_X = pad_sequences(test_X, maxlen=max_len, padding="post", truncating="post")
    
# #     # 把最常用的40个词去掉，pad为0
# #     train_X = np.where(train_X>=40, train_X, 0)
# #     test_X = np.where(test_X>=40, test_X, 0)
    
    #shuffling the data
    np.random.seed(2019)
    trn_idx = np.random.permutation(len(train_X))

    train_X = train_X[trn_idx]
    train_Y = train_Y[trn_idx]
    
    # 使用本地测试集
    if use_local_test:
        train_X, local_test_X = (train_X[:-2*len(test_X)], train_X[-2*len(test_X):])
        train_Y, local_test_Y = (train_Y[:-2*len(test_X)], train_Y[-2*len(test_X):])
    else:
        local_test_X = np.zeros(shape=[1,max_len], dtype=np.int32)
        local_test_Y = np.zeros(shape=[1], dtype=np.int32)
        
    print(train_X.shape)
    print(local_test_X.shape)
    print(test_X.shape)
    print(len(tokenizer.word_index))
    
    return train_X, test_X, train_Y, local_test_X, local_test_Y, tokenizer.word_index

# load_and_prec()

# load embeddings

In [6]:
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

def load_fasttext(word_index):
    """
    这个加载词向量还没有细看
    """
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100 and o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

# Utils

In [26]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.eager import context

def cyclic_learning_rate(global_step,
                         learning_rate=0.001,
                         max_lr=0.004,
                         step_size=20.,
                         gamma=0.99994,
                         mode='triangular',
                         name=None):
  if global_step is None:
    raise ValueError("global_step is required for cyclic_learning_rate.")
  with ops.name_scope(name, "CyclicLearningRate",
                      [learning_rate, global_step]) as name:
    learning_rate = ops.convert_to_tensor(learning_rate, name="learning_rate")
    dtype = learning_rate.dtype
    global_step = math_ops.cast(global_step, dtype)
    step_size = math_ops.cast(step_size, dtype)
    def cyclic_lr():
      """Helper to recompute learning rate; most helpful in eager-mode."""
      # computing: cycle = floor( 1 + global_step / ( 2 * step_size ) )
      double_step = math_ops.multiply(2., step_size)
      global_div_double_step = math_ops.divide(global_step, double_step)
      cycle = math_ops.floor(math_ops.add(1., global_div_double_step))
      # computing: x = abs( global_step / step_size – 2 * cycle + 1 )
      double_cycle = math_ops.multiply(2., cycle)
      global_div_step = math_ops.divide(global_step, step_size)
      tmp = math_ops.subtract(global_div_step, double_cycle)
      x = math_ops.abs(math_ops.add(1., tmp))
      # computing: clr = learning_rate + ( max_lr – learning_rate ) * max( 0, 1 - x )
      a1 = math_ops.maximum(0., math_ops.subtract(1., x))
      a2 = math_ops.subtract(max_lr, learning_rate)
      clr = math_ops.multiply(a1, a2)
      if mode == 'triangular2':
        clr = math_ops.divide(clr, math_ops.cast(math_ops.pow(2, math_ops.cast(
            cycle-1, tf.int32)), tf.float32))
      if mode == 'exp_range':
        clr = math_ops.multiply(math_ops.pow(gamma, global_step), clr)
      return math_ops.add(clr, learning_rate, name=name)
    if not context.executing_eagerly():
      cyclic_lr = cyclic_lr()
    return cyclic_lr

In [8]:
# dense layer
def dense(inputs, hidden, use_bias=True, 
          w_initializer=tf.contrib.layers.xavier_initializer(), b_initializer=tf.constant_initializer(0.1), scope="dense"):
    """
    全连接层
    """
    with tf.variable_scope(scope):
        shape = tf.shape(inputs)
        dim = inputs.get_shape().as_list()[-1]
        out_shape = [shape[idx] for idx in range(
            len(inputs.get_shape().as_list()) - 1)] + [hidden]
        # 如果是三维的inputs，reshape成二维
        flat_inputs = tf.reshape(inputs, [-1, dim])
        W = tf.get_variable("W", [dim, hidden], initializer=w_initializer)
        res = tf.matmul(flat_inputs, W)
        if use_bias:
            b = tf.get_variable("b", [hidden], initializer=b_initializer)
            res = tf.nn.bias_add(res, b)
        # outshape就是input的最后一维变成hidden
        res = tf.reshape(res, out_shape)
        return res

In [9]:
# dot-product attention
def dot_attention(inputs, memory, mask, hidden, keep_prob, scope="dot_attention"):
    """
    门控attention层
    """
    def softmax_mask(val, mask):
        return -1e30 * (1 - tf.cast(mask, tf.float32)) + val
    
    with tf.variable_scope(scope):
        JX = tf.shape(inputs)[1]  # inputs的1维度，应该是c_maxlen
        with tf.variable_scope("attention"):
            # inputs_的shape:[batch_size, c_maxlen, hidden]
            inputs_ = tf.nn.relu(
                dense(inputs, hidden, use_bias=False, scope="inputs"))
            memory_ = tf.nn.relu(
                dense(memory, hidden, use_bias=False, scope="memory"))
            # 三维矩阵相乘，结果的shape是[batch_size, c_maxlen, q_maxlen]
            outputs = tf.matmul(inputs_, tf.transpose(
                memory_, [0, 2, 1])) / (hidden ** 0.5)
            # 将mask平铺成与outputs相同的形状，这里考虑，改进成input和memory都需要mask
            mask = tf.tile(tf.expand_dims(mask, axis=1), [1, JX, 1])
            logits = tf.nn.softmax(softmax_mask(outputs, mask))
            outputs = tf.matmul(logits, memory)
            # res:[batch_size, c_maxlen, 12*hidden]
            res = tf.concat([inputs, outputs], axis=2)
            return res

#         with tf.variable_scope("gate"):
#             """
#             attention * gate
#             """
#             dim = res.get_shape().as_list()[-1]
#             d_res = dropout(res, keep_prob=keep_prob, is_train=is_train)
#             gate = tf.nn.sigmoid(dense(d_res, dim, use_bias=False))
#             return res * gate  # 向量的逐元素相乘

In [10]:
# 定义一个多层的双向rnn类，使用cudnn加速, 包括lstm和gru。
class cudnn_rnn:
    def __init__(self, num_layers, num_units, input_size, neuron="GRU", scope=None):
        self.num_layers = num_layers
        self.rnns = []
        self.scope = scope
        self.neuron = neuron
        for layer in range(num_layers):
            input_size_ = input_size if layer == 0 else 2 * num_units
            if self.neuron == "GRU":
                rnn_fw = tf.contrib.cudnn_rnn.CudnnGRU(1, num_units, name="f_cudnn_gru")
                rnn_bw = tf.contrib.cudnn_rnn.CudnnGRU(1, num_units, name="b_cudnn_gru")
            elif self.neuron == "LSTM":
                rnn_fw = tf.contrib.cudnn_rnn.CudnnLSTM(1, num_units, name="f_cudnn_lstm")
                rnn_bw = tf.contrib.cudnn_rnn.CudnnLSTM(1, num_units, name="b_cudnn_lstm")
            else:
                raise NameError
            self.rnns.append((rnn_fw, rnn_bw, ))

    def __call__(self, inputs, seq_len, keep_prob, concat_layers=True):
        # cudnn GRU需要交换张量的维度，可能是便于计算
        outputs = [tf.transpose(inputs, [1, 0, 2])]
        out_states = []
        with tf.variable_scope(self.scope):
            for layer in range(self.num_layers):
                rnn_fw, rnn_bw = self.rnns[layer]
                with tf.variable_scope("fw_{}".format(layer)):
                    if self.neuron == "GRU":
                        out_fw, (fw_state,) = rnn_fw(outputs[-1])
                    else:
                        out_fw, (fw_state,_) = rnn_fw(outputs[-1])
                with tf.variable_scope("bw_{}".format(layer)):
                    inputs_bw = tf.reverse_sequence(outputs[-1], seq_lengths=seq_len, seq_dim=0, batch_dim=1)
                    if self.neuron == "GRU":
                        out_bw, (bw_state,) = rnn_bw(outputs[-1])
                    else:
                        out_bw, (bw_state,_) = rnn_bw(outputs[-1])
                    out_bw = tf.reverse_sequence(out_bw, seq_lengths=seq_len, seq_dim=0, batch_dim=1)
                outputs.append(tf.concat([out_fw, out_bw], axis=2))
                out_states.append(tf.concat([fw_state, bw_state], axis=-1))
        if concat_layers:
            res = tf.concat(outputs[1:], axis=2)
            final_state = tf.squeeze(tf.transpose(tf.concat(out_states, axis=0), [1,0,2]), axis=1)
        else:
            res = outputs[-1]
            final_state = tf.squeeze(out_states[-1], axis=0)
        res = tf.transpose(res, [1, 0, 2])
        return res, final_state

# Models

text_cnn

In [27]:
class model_text_cnn(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(self, embedding_matrix, sequence_length=50, num_classes=1,
                 embedding_size=300, filter_sizes=[2,3,5], num_filters=256, trainable=True):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.int32, [None], name="input_y")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        
        # Some variables
        self.embedding_matrix = tf.get_variable("embedding_matrix", initializer=tf.constant(
                embedding_matrix, dtype=tf.float32), trainable=False)
        self.global_step = tf.get_variable('global_step', shape=[], dtype=tf.int32,
                                           initializer=tf.constant_initializer(0), trainable=False)
        
        # The structure of the model
        self.layers(num_classes, filter_sizes, num_filters)
        # optimizer
        if trainable:
#             self.learning_rate = tf.train.exponential_decay(
#                 learning_rate=0.0015, global_step=self.global_step, decay_steps=1000, decay_rate=0.95)
            self.learning_rate = cyclic_learning_rate(
                global_step=self.global_step,
                step_size=2000)
            self.opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate, epsilon=1e-8)
            self.train_op = self.opt.minimize(self.loss, global_step=self.global_step)
    
    def layers(self, num_classes, filter_sizes, num_filters):
        # Embedding layer
        with tf.variable_scope("embedding"):
            self.embedding_inputs = tf.nn.embedding_lookup(self.embedding_matrix, self.input_x)
            self.embedding_inputs = tf.nn.dropout(self.embedding_inputs, self.keep_prob)
        
        with tf.variable_scope("convolutional_layers"):
            """
            卷积和池化操作
            """
            self.pooled_outputs = []
            for i, filter_size in enumerate(filter_sizes):
                with tf.variable_scope("conv-maxpool-{}".format(filter_size)):
                    # conv
                    filter_W = tf.get_variable(shape=[filter_size, embedding_size, num_filters],
                                               initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                               name="filter_W", dtype=tf.float32)
                    filter_b = tf.get_variable(shape=[num_filters], name="filter_b", dtype=tf.float32)
                    conv = tf.nn.conv1d(
                        value=self.embedding_inputs,
                        filters=filter_W,
                        stride=1,
                        padding="VALID",
                        name="conv") # shape:[batch_size, seq_len-filter_size+1, num_filters]
                    conv = tf.nn.bias_add(conv, filter_b)
                    
#                     # batch normalization
#                     conv = tf.contrib.layers.batch_norm(conv)
                    
                    # relu
                    h = tf.nn.relu(conv, name="relu")
#                     # 原始论文的max-pooling层，只保留1个最大值的信息，就相当于reduce_max
#                     pooled_out = tf.reduce_max(h, axis=1, name='global_max_pooling') # shape: [batch_size, num_filters]
                    # 使用一个的max-ave-pooling
                    pooled = tf.nn.max_pool(
                        tf.expand_dims(h[:,:45,:], axis=-1),
                        ksize=[1,5,1,1],
                        strides=[1,5,1,1],
                        padding="VALID",
                        name="max_pool") # shape: [batch_size, 9, num_filters, 1]
                    pooled_out = tf.reduce_mean(tf.squeeze(pooled, axis=-1), axis=1)
                    self.pooled_outputs.append(pooled_out)
        
            # Combine all the pooled features
            self.conv_outputs = tf.concat(self.pooled_outputs, axis=1)
            # dropout
            self.conv_outputs_drop = tf.nn.dropout(self.conv_outputs, self.keep_prob)
        
        with tf.variable_scope("fully_connected"):
            """
            全连接层
            """
            fc_1 = dense(inputs=self.conv_outputs_drop, hidden=128, use_bias=True, scope="FC_1")
            fc_1 = tf.nn.relu(fc_1)
            fc_1_drop = tf.nn.dropout(fc_1, self.keep_prob)
            fc_2 = dense(inputs=fc_1_drop, hidden=num_classes, use_bias=True, scope="FC_2")
            self.logits = tf.squeeze(fc_2, name="logits")
        
        with tf.variable_scope("sigmoid_and_loss"):
            """
            用sigmoid函数加阈值代替softmax的多分类
            """
            self.sigmoid = tf.nn.sigmoid(self.logits)
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=self.logits, labels=tf.cast(self.input_y, dtype=tf.float32)))
 

text_rnn

In [28]:
class model_text_rnn(object):
    """
    使用简单的双向GRU实现分类。
    """
    def __init__(self, embedding_matrix, sequence_length=50, num_classes=1,
                 embedding_size=300, trainable=True):
        
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.int32, [None], name="input_y")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        
        # Some variables
        self.embedding_matrix = tf.get_variable("embedding_matrix", initializer=tf.constant(
                embedding_matrix, dtype=tf.float32), trainable=False)
        self.global_step = tf.get_variable('global_step', shape=[], dtype=tf.int32,
                                           initializer=tf.constant_initializer(0), trainable=False)
        
        with tf.name_scope("process"):
            self.seq_len = tf.reduce_sum(tf.cast(tf.cast(self.input_x, dtype=tf.bool), dtype=tf.int32), axis=1, name="seq_len")
        
        # The structure of the model
        self.layers(num_classes)
        # optimizer
        if trainable:
#             self.learning_rate = tf.train.exponential_decay(
#                 learning_rate=0.0015, global_step=self.global_step, decay_steps=1000, decay_rate=0.95)
            self.learning_rate = cyclic_learning_rate(
                global_step=self.global_step,
                step_size=2000)
            self.opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate, epsilon=1e-8)
            self.train_op = self.opt.minimize(self.loss, global_step=self.global_step)
        
    def layers(self, num_classes):
        # Embedding layer
        with tf.variable_scope("embedding"):
            self.embedding_inputs = tf.nn.embedding_lookup(self.embedding_matrix, self.input_x)
            self.embedding_inputs = tf.nn.dropout(self.embedding_inputs, self.keep_prob)
        
        # Bi-GRU Encoder
        with tf.variable_scope("Bi-GRU"):
            bi_rnn = cudnn_rnn(num_layers=1, num_units=128, input_size=self.embedding_inputs.get_shape().as_list()[-1], scope="encoder")
            _, final_state = bi_rnn(self.embedding_inputs, seq_len=self.seq_len, keep_prob=self.keep_prob) 
            # shape: [batch_size, 2*hidden]
            self.rnn_out = tf.nn.dropout(final_state, keep_prob=self.keep_prob)
            
        with tf.variable_scope("fully_connected"):
            """
            全连接层
            """
            fc_1 = dense(inputs=self.rnn_out, hidden=128, use_bias=True, scope="FC_1")
            fc_1 = tf.nn.relu(fc_1)
            fc_1_drop = tf.nn.dropout(fc_1, self.keep_prob)
            
            fc_2 = dense(inputs=fc_1_drop, hidden=num_classes, use_bias=True, scope="FC_2")
            self.logits = tf.squeeze(fc_2, name="logits")
        
        with tf.variable_scope("sigmoid_and_loss"):
            """
            用sigmoid函数加阈值代替softmax的多分类
            """
            self.sigmoid = tf.nn.sigmoid(self.logits)
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=self.logits, labels=tf.cast(self.input_y, dtype=tf.float32)))
            

test_rnn + attention

In [29]:
class model_text_rnn_attention(object):
    """
    使用简单的双向GRU,并接一个attention层。
    """
    def __init__(self, embedding_matrix, sequence_length=50, num_classes=1,
                 embedding_size=300, trainable=True):
        
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.int32, [None], name="input_y")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        
        # Some variables
        self.embedding_matrix = tf.get_variable("embedding_matrix", initializer=tf.constant(
                embedding_matrix, dtype=tf.float32), trainable=False)
        self.global_step = tf.get_variable('global_step', shape=[], dtype=tf.int32,
                                           initializer=tf.constant_initializer(0), trainable=False)
        
        with tf.name_scope("process"):
            self.seq_len = tf.reduce_sum(tf.cast(tf.cast(self.input_x, dtype=tf.bool), dtype=tf.int32), axis=1, name="seq_len")
            self.mask = tf.cast(self.input_x, dtype=tf.bool)
        
        # The structure of the model
        self.layers(num_classes)
        # optimizer
        if trainable:
#             self.learning_rate = tf.train.exponential_decay(
#                 learning_rate=0.0015, global_step=self.global_step, decay_steps=1000, decay_rate=0.95)
            self.learning_rate = cyclic_learning_rate(
                global_step=self.global_step,
                step_size=2000)
            self.opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate, epsilon=1e-8)
            self.train_op = self.opt.minimize(self.loss, global_step=self.global_step)
        
    def layers(self, num_classes):
        # Embedding layer
        with tf.variable_scope("embedding"):
            self.embedding_inputs = tf.nn.embedding_lookup(self.embedding_matrix, self.input_x)
            self.embedding_inputs = tf.nn.dropout(self.embedding_inputs, self.keep_prob)
        
        # Bi-GRU Encoder
        with tf.variable_scope("Bi-GRU"):
            bi_rnn = cudnn_rnn(num_layers=1, num_units=128, input_size=self.embedding_inputs.get_shape().as_list()[-1], scope="encoder")
            rnn_output, _ = bi_rnn(self.embedding_inputs, seq_len=self.seq_len, keep_prob=self.keep_prob) 
            # shape: [batch_size, 2*hidden]
            self.rnn_out = tf.nn.dropout(rnn_output, keep_prob=self.keep_prob)
        
        with tf.variable_scope("Attention_Layer"):
            """
            将rnn的输出再做self-attention
            """
            att = dot_attention(inputs=self.rnn_out, memory=self.rnn_out, mask=self.mask, hidden=128,
                                keep_prob=self.keep_prob)
            # pooling
            att_out_1 = tf.reduce_mean(att, axis=2) # shape: [batch_size, 50]
            att_out_2 = tf.reduce_max(att, axis=2)
            self.att_out = tf.concat([att_out_1, att_out_2], axis=1)
            self.att_out = tf.nn.dropout(self.att_out, self.keep_prob)
            
        with tf.variable_scope("fully_connected"):
            """
            全连接层
            """
            fc = dense(inputs=self.att_out, hidden=num_classes, use_bias=True, scope="FC")
            self.logits = tf.squeeze(fc, name="logits")
        
        with tf.variable_scope("sigmoid_and_loss"):
            """
            用sigmoid函数加阈值代替softmax的多分类
            """
            self.sigmoid = tf.nn.sigmoid(self.logits)
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=self.logits, labels=tf.cast(self.input_y, dtype=tf.float32)))
            

text_rcnn

In [30]:
class model_text_rcnn(object):
    """
    使用简单的双向GRU,将词的表示扩展为（上文+embedding+下文）的形式，后面再接1-卷积和pooling，也优不接卷积直接pooling的实现。
    """
    def __init__(self, embedding_matrix, sequence_length=50, num_classes=1,
                 embedding_size=300, trainable=True):
        
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.int32, [None], name="input_y")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        
        # Some variables
        self.embedding_matrix = tf.get_variable("embedding_matrix", initializer=tf.constant(
                embedding_matrix, dtype=tf.float32), trainable=False)
        self.global_step = tf.get_variable('global_step', shape=[], dtype=tf.int32,
                                           initializer=tf.constant_initializer(0), trainable=False)
        
        with tf.name_scope("process"):
            self.seq_len = tf.reduce_sum(tf.cast(tf.cast(self.input_x, dtype=tf.bool), dtype=tf.int32), axis=1, name="seq_len")
            self.mask = tf.cast(self.input_x, dtype=tf.bool)
        
        # The structure of the model
        self.layers(num_classes)
        # optimizer
        if trainable:
#             self.learning_rate = tf.train.exponential_decay(
#                 learning_rate=0.0015, global_step=self.global_step, decay_steps=1000, decay_rate=0.95)
            self.learning_rate = cyclic_learning_rate(
                global_step=self.global_step,
                step_size=2000)
            self.opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate, epsilon=1e-8)
            self.train_op = self.opt.minimize(self.loss, global_step=self.global_step)
        
    def layers(self, num_classes):
        # Embedding layer
        with tf.variable_scope("embedding"):
            self.embedding_inputs = tf.nn.embedding_lookup(self.embedding_matrix, self.input_x)
            self.embedding_inputs = tf.nn.dropout(self.embedding_inputs, self.keep_prob)
        
        # Bi-GRU Encoder
        with tf.variable_scope("Bi-GRU"):
            bi_rnn = cudnn_rnn(num_layers=1, num_units=128, input_size=self.embedding_inputs.get_shape().as_list()[-1], scope="encoder")
            rnn_output, _ = bi_rnn(self.embedding_inputs, seq_len=self.seq_len, keep_prob=self.keep_prob) 
            self.rnn_out = tf.nn.dropout(rnn_output, keep_prob=self.keep_prob)
        
        with tf.variable_scope("word_representation"):
            """
            词的上下文表示
            """
            cut = int(self.rnn_out.get_shape().as_list()[-1] / 2)
            left = self.rnn_out[:,:,:cut]
            right = self.rnn_out[:,:,cut:]
            self.word_out = tf.concat([left, self.embedding_inputs, right], axis=2) # shape: [batch_size, 50, 2*128+300]
        
        with tf.variable_scope("text_representation"):
            """
            text的表示，可以直接线性层加max-pooling，也可以加卷积
            """
            self.text_out = tf.tanh(dense(self.word_out, 128))
            self.pool_out = tf.reduce_max(self.text_out, axis=1)
            self.pool_out = tf.nn.dropout(self.pool_out, self.keep_prob)
            
        with tf.variable_scope("fully_connected"):
            """
            全连接层
            """
            fc = dense(inputs=self.pool_out, hidden=num_classes, use_bias=True, scope="FC")
            self.logits = tf.squeeze(fc, name="logits")
        
        with tf.variable_scope("sigmoid_and_loss"):
            """
            用sigmoid函数加阈值代替softmax的多分类
            """
            self.sigmoid = tf.nn.sigmoid(self.logits)
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=self.logits, labels=tf.cast(self.input_y, dtype=tf.float32)))
            

LSTM & GRU

In [31]:
class model_lstm_gru(object):
    """
    使用简单的双向GRU实现分类。
    """
    def __init__(self, embedding_matrix, sequence_length=50, num_classes=1,
                 embedding_size=300, trainable=True):
        
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.int32, [None], name="input_y")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        
        # Some variables
        self.embedding_matrix = tf.get_variable("embedding_matrix", initializer=tf.constant(
                embedding_matrix, dtype=tf.float32), trainable=False)
        self.global_step = tf.get_variable('global_step', shape=[], dtype=tf.int32,
                                           initializer=tf.constant_initializer(0), trainable=False)
        
        with tf.name_scope("process"):
            self.seq_len = tf.reduce_sum(tf.cast(tf.cast(self.input_x, dtype=tf.bool), dtype=tf.int32), axis=1, name="seq_len")
            self.mask = tf.cast(self.input_x, dtype=tf.bool)
        
        # The structure of the model
        self.layers(num_classes)
        # optimizer
        if trainable:
#             self.learning_rate = tf.train.exponential_decay(
#                 learning_rate=0.0015, global_step=self.global_step, decay_steps=1000, decay_rate=0.95)
            self.learning_rate = cyclic_learning_rate(
                global_step=self.global_step,
                step_size=2000)
            self.opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate, epsilon=1e-8)
            self.train_op = self.opt.minimize(self.loss, global_step=self.global_step)
        
    def layers(self, num_classes):
        # Embedding layer
        with tf.variable_scope("embedding"):
            self.embedding_inputs = tf.nn.embedding_lookup(self.embedding_matrix, self.input_x)
            self.embedding_inputs = tf.nn.dropout(self.embedding_inputs, self.keep_prob)
        
        # Bi-RNN Encoder
        with tf.variable_scope("Bi-RNN"):
            # LSTM
            bi_lstm = cudnn_rnn(
                num_layers=1, num_units=64, input_size=self.embedding_inputs.get_shape().as_list()[-1], neuron="LSTM", scope="LSTM")
            self.lstm_out, _ = bi_lstm(self.embedding_inputs, seq_len=self.seq_len, keep_prob=self.keep_prob)
            self.lstm_out = tf.nn.dropout(self.lstm_out, keep_prob=self.keep_prob)
            # GRU
            bi_gru = cudnn_rnn(num_layers=1, num_units=64, input_size=self.lstm_out.get_shape().as_list()[-1], scope="GRU")
            self.gru_out, _ = bi_gru(self.lstm_out, seq_len=self.seq_len, keep_prob=self.keep_prob) 
            self.gru_out = tf.nn.dropout(self.gru_out, keep_prob=self.keep_prob)
        
        with tf.variable_scope("double_attention"):
            lstm_att = dot_attention(
                inputs=self.lstm_out, memory=self.lstm_out, mask=self.mask, hidden=128, keep_prob=self.keep_prob, scope="l_att")
            gru_att = dot_attention(
                inputs=self.gru_out, memory=self.gru_out, mask=self.mask, hidden=128, keep_prob=self.keep_prob, scope="g_att")
            # pooling
            att_out_lstm = tf.reduce_mean(lstm_att, axis=1) # shape: [batch_size, 256]
            att_out_gru = tf.reduce_max(gru_att, axis=1)
            self.att_out = tf.concat([att_out_lstm, att_out_gru], axis=1)
            
        with tf.variable_scope("fully_connected"):
            """
            全连接层
            """
            fc_1 = dense(inputs=self.att_out, hidden=64, use_bias=True, scope="FC_1")
            fc_1 = tf.nn.relu(fc_1)
            fc_1_drop = tf.nn.dropout(fc_1, self.keep_prob)
            
            fc_2 = dense(inputs=fc_1_drop, hidden=num_classes, use_bias=True, scope="FC_2")
            self.logits = tf.squeeze(fc_2, name="logits")
        
        with tf.variable_scope("sigmoid_and_loss"):
            """
            用sigmoid函数加阈值代替softmax的多分类
            """
            self.sigmoid = tf.nn.sigmoid(self.logits)
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=self.logits, labels=tf.cast(self.input_y, dtype=tf.float32)))
            

fast_text

In [16]:
class model_fastText(object):
    """
    将词向量平均，然后直接全连接层分类。
    """
    def __init__(self, embedding_matrix, sequence_length=50, num_classes=1,
                 embedding_size=300, trainable=True):
        
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.int32, [None], name="input_y")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        
        # Some variables
        self.embedding_matrix = tf.get_variable("embedding_matrix", initializer=tf.constant(
                embedding_matrix, dtype=tf.float32), trainable=False)
        self.global_step = tf.get_variable('global_step', shape=[], dtype=tf.int32,
                                           initializer=tf.constant_initializer(0), trainable=False)
        
        with tf.name_scope("process"):
            self.seq_len = tf.reduce_sum(tf.cast(tf.cast(self.input_x, dtype=tf.bool), dtype=tf.int32), axis=1, name="seq_len")
            self.mask = tf.cast(self.input_x, dtype=tf.bool)
        
        # The structure of the model
        self.layers(num_classes)
        # optimizer
        if trainable:
            self.learning_rate = tf.train.exponential_decay(
                learning_rate=0.001, global_step=self.global_step, decay_steps=2000, decay_rate=0.95)
            self.opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate, epsilon=1e-8)
            self.train_op = self.opt.minimize(self.loss, global_step=self.global_step)
        
    def layers(self, num_classes):
        # Embedding layer
        with tf.variable_scope("embedding"):
            self.embedding_inputs = tf.nn.embedding_lookup(self.embedding_matrix, self.input_x)
#             self.embedding_inputs = tf.nn.dropout(self.embedding_inputs, self.keep_prob)
        
        with tf.variable_scope("pooling"):
            """
            text的表示，直接将词向量average-pooling
            """
            self.pool_out = tf.reduce_mean(self.embedding_inputs, axis=1)
            
        with tf.variable_scope("fully_connected"):
            """
            全连接层
            """
            fc_1 = dense(inputs=self.pool_out, hidden=512, use_bias=True, scope="FC_1")
            fc_1 = tf.nn.relu(fc_1)
            fc_1_drop = tf.nn.dropout(fc_1, self.keep_prob)
            
            fc_2 = dense(inputs=fc_1_drop, hidden=128, use_bias=True, scope="FC_2")
            fc_2 = tf.nn.relu(fc_2)
            fc_2_drop = tf.nn.dropout(fc_2, self.keep_prob)
            
            fc_3 = dense(inputs=fc_2_drop, hidden=num_classes, use_bias=True, scope="FC_3")
            self.logits = tf.squeeze(fc_3, name="logits")
        
        with tf.variable_scope("sigmoid_and_loss"):
            """
            用sigmoid函数加阈值代替softmax的多分类
            """
            self.sigmoid = tf.nn.sigmoid(self.logits)
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    logits=self.logits, labels=tf.cast(self.input_y, dtype=tf.float32)))
            

# Training Tools

In [17]:
# batch生成器
def batch_generator(train_X, train_Y, batch_size, is_train=True, seed=1234):
    """
    batch生成器:
    在is_train为true的情况下，补充batch，并shuffle
    """
    data_number = train_X.shape[0]
    batch_count = 0
    while True:
        if batch_count * batch_size + batch_size > data_number:
            # 最后一个batch的操作
            if is_train:
                # 后面的直接舍弃，重新开始
                # shuffle
                np.random.seed(seed)
                trn_idx = np.random.permutation(data_number)
                train_X = train_X[trn_idx]
                train_Y = train_Y[trn_idx]
                one_batch_X = train_X[0:batch_size]
                one_batch_Y = train_Y[0:batch_size]
                batch_count = 1
                yield one_batch_X, one_batch_Y
            else:
                one_batch_X = train_X[batch_count * batch_size:data_number]
                one_batch_Y = train_Y[batch_count * batch_size:data_number]
                batch_count = 0
                yield one_batch_X, one_batch_Y
        else:
            one_batch_X = train_X[batch_count * batch_size:batch_count * batch_size + batch_size]
            one_batch_Y = train_Y[batch_count * batch_size:batch_count * batch_size + batch_size]
            batch_count += 1
            yield one_batch_X, one_batch_Y


In [18]:
# 正类欠采样，负类数据增强，暂时用随机打乱数据增强.
def data_augmentation(X, Y, under_sample=100000, aug_num=3):
    """
    under_sample: 欠采样个数
    aug: 数据增强倍数
    """
    pos_X = []
    neg_X = []
    for i in range(X.shape[0]):
        if Y[i] == 1:
            neg_X.append(list(X[i]))
        else:
            pos_X.append(list(X[i]))
    
    # 正样本欠采样
    random.shuffle(pos_X)
    pos_X = pos_X[:-under_sample]
    
    # 正样本数据增强
    pos_X_aug = []
    for i in range(200000):
        aug = []
        for x in pos_X[i]:
            if x != 0:
                aug.append(x)
            else:
                break
        random.shuffle(aug)
        aug += [0] * (max_len-len(aug))
        pos_X_aug.append(aug)
    pos_X.extend(pos_X_aug)
    print(len(pos_X))
    
    # 负样本数据增强
    neg_X_aug = []
    for i in range(aug_num):
        for neg in neg_X:
            aug = []
            for x in neg:
                if x != 0:
                    aug.append(x)
                else:
                    break
            random.shuffle(aug)
            aug += [0] * (max_len-len(aug))
            neg_X_aug.append(aug)
        
    neg_X.extend(neg_X_aug)
    print(len(neg_X))
    
    pos_Y = np.zeros(shape=[len(pos_X)], dtype=np.int32)
    neg_Y = np.ones(shape=[len(neg_X)], dtype=np.int32)
    
    pos_X.extend(neg_X)
    X_out = np.array(pos_X, dtype=np.int32)
    Y_out = np.append(pos_Y, neg_Y)
    
    print(X_out.shape)
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(X_out))

    X_out = X_out[trn_idx]
    Y_out = Y_out[trn_idx]
    
    print(X_out.shape)
    print(Y_out.shape)

    return X_out, Y_out

In [19]:
# 搜索最佳阈值
def bestThreshold(y,y_preds):
    tmp = [0,0,0] # idx, cur, max
    delta = 0
    for tmp[0] in tqdm(np.arange(0.1, 0.501, 0.01)):
        tmp[1] = metrics.f1_score(y, np.array(y_preds)>tmp[0])
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))
    return delta , tmp[2]


# Seed

In [20]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True

# Main part

In [21]:
# 加载数据，平均词向量
train_X, test_X, train_Y, local_test_X, local_test_Y, word_index = load_and_prec(use_local_test)

seed_everything()
embedding_matrix_1 = load_glove(word_index)
embedding_matrix_2 = load_fasttext(word_index)
embedding_matrix_3 = load_para(word_index)
embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_2, embedding_matrix_3], axis = 0)
np.shape(embedding_matrix)
# embedding_matrix = np.zeros(shape=[100,300],dtype=np.float32)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


,qid,question_text
0,00014894849d00ba98a9,My voice range is A2-C5. My chest voice goes u...
1,000156468431f09b3cae,How much does a tutor earn in Bangalore?
2,000227734433360e1aae,What are the best made pocket knives under $20...
3,0005e06fbe3045bd2a92,Why would they add a hypothetical scenario tha...
4,00068a0f7f41f50fc399,What is the dresscode for Techmahindra freshers?


80810
(1193382, 50)
(112740, 50)
(56370, 50)
185946


(120000, 300)

In [32]:
# 多折训练，交叉验证平均，测试

# 随机种子
SEED = 6017

# 划分交叉验证集
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(train_X, train_Y))

# test batch
test_batch = batch_generator(test_X, np.zeros(shape=[test_X.shape[0]], dtype=np.int32), batch_size, False)
local_test_batch = batch_generator(local_test_X, local_test_Y, batch_size, False)

# 最终输出
train_preds = np.zeros(len(train_X), dtype=np.float32)
test_preds = np.zeros((len(test_X), len(splits)), dtype=np.float32)
test_preds_local = np.zeros((len(local_test_X), len(splits)), dtype=np.float32)

# 多折训练
for i, (train_idx, valid_idx) in enumerate(splits):
    print("fold:{}".format(i+1))
    start_time = time.time()
    
    X_train = train_X[train_idx]
    Y_train = train_Y[train_idx]
    X_val = train_X[valid_idx]
    Y_val = train_Y[valid_idx]
    
#     # 数据增强
#     X_train, Y_train = data_augmentation(X_train, Y_train) 
#     print(Y_train[:100])
#     print(Y_train[-100:])
    
    # 训练batch生成器
    train_batch = batch_generator(X_train, Y_train, batch_size, True, SEED+i)
    val_batch = batch_generator(X_val, Y_val, batch_size, False)
    
    # 选择最好的结果
    best_val_f1 = 0.0
    best_val_loss = 99999.99999
    best_val_fold = []
    best_test_fold = []
    best_local_test_fold = []
    
    # 训练 & 验证 & 测试
    with tf.Graph().as_default():
        sess_config = tf.ConfigProto(allow_soft_placement=True)
        sess_config.gpu_options.allow_growth = True
        with tf.Session(config=sess_config) as sess:
            
            # seed
            seed_everything(SEED+i)
            tf.set_random_seed(SEED+i)
            
            # 模型选择， 每一折用不同模型
            if i==0:
                model = model_text_cnn(embedding_matrix=embedding_matrix, sequence_length=max_len)
                KP = 0.7
                num_steps = 16000
                print_steps = 10000
            elif i==1:
                model = model_text_rnn(embedding_matrix=embedding_matrix, sequence_length=max_len)
                KP = 0.7
                num_steps = 16000
                print_steps = 10000
            elif i==2:
                model = model_text_rnn_attention(embedding_matrix=embedding_matrix, sequence_length=max_len)
                KP = 0.7
                num_steps = 16000
                print_steps = 10000
            elif i==3:
                model = model_text_rcnn(embedding_matrix=embedding_matrix, sequence_length=max_len)
                KP = 0.7
                num_steps = 18000
                print_steps = 10000
            else:
                model = model_lstm_gru(embedding_matrix=embedding_matrix, sequence_length=max_len)
                KP = 0.7
                num_steps = 16000
                print_steps = 10000
            
            sess.run(tf.global_variables_initializer())
            
            train_loss_sum = 0.0
            for go in range(num_steps):
                steps = sess.run(model.global_step) + 1
                
                # 训练
                train_batch_X, train_batch_Y = next(train_batch)
                feed = {model.input_x:train_batch_X, model.input_y:train_batch_Y, model.keep_prob:KP}
                loss, train_op = sess.run([model.loss, model.train_op], feed_dict=feed)
                train_loss_sum += loss
                
                # 验证 & 测试
                if steps % 1000 == 0 and steps >= print_steps: 
                    val_predictions = []
                    val_loss_sum = 0.0
                    for _ in range(X_val.shape[0] // batch_size + 1):
                        val_batch_X, val_batch_Y = next(val_batch)
                        feed_val = {model.input_x:val_batch_X, model.input_y:val_batch_Y, model.keep_prob:1.0}
                        val_loss, val_sigmoid = sess.run([model.loss, model.sigmoid], feed_dict=feed_val)
                        val_predictions.extend(val_sigmoid)
                        val_loss_sum += val_loss
                    val_loss_sum = val_loss_sum / (X_val.shape[0] // batch_size + 1)
                    
                    print("steps:{}, train_loss:{:.5f}, val_loss:{:.5f}".format(
                        steps, float(train_loss_sum / 1000), float(val_loss_sum)))
                    
                    # train loss
                    train_loss_sum = 0.0
                    
                    # 测试，并选取最低的loss值的时刻的测试结果为最终结果
#                     if val_loss_sum < best_val_loss:
                    if steps == 16000:
                        best_val_loss = val_loss_sum
                        best_val_fold = val_predictions
                        best_test_fold = []
                        best_local_test_fold = []
                        # 线上test
                        for _ in range(test_X.shape[0] // batch_size + 1):
                            test_batch_X, _ = next(test_batch)
                            feed_test = {model.input_x:test_batch_X, model.keep_prob:1.0}
                            test_sigmoid = sess.run(model.sigmoid, feed_dict=feed_test)
                            best_test_fold.extend(test_sigmoid)
                        # 线下test
                        if use_local_test:
                            for _ in range(local_test_X.shape[0] // batch_size + 1):
                                local_test_batch_X, _ = next(local_test_batch)
                                feed_local_test = {model.input_x:local_test_batch_X, model.keep_prob:1.0}
                                local_test_sigmoid = sess.run(model.sigmoid, feed_dict=feed_local_test)
                                best_local_test_fold.extend(local_test_sigmoid)
                        print("test done!")
                
    
    # 更新预测结果
    train_preds[valid_idx] = np.array(best_val_fold)
    test_preds[:, i] = np.array(best_test_fold)
    if use_local_test:
        test_preds_local[:, i] = np.array(best_local_test_fold)
    
    end_time = time.time()
    print("The time of fold {} is: {:.5f}s.".format(i+1, end_time-start_time))


fold:1
steps:10000, train_loss:1.11889, val_loss:0.10480
steps:11000, train_loss:0.10298, val_loss:0.10248
steps:12000, train_loss:0.09627, val_loss:0.10355
steps:13000, train_loss:0.09568, val_loss:0.10280
steps:14000, train_loss:0.09538, val_loss:0.10605
steps:15000, train_loss:0.09848, val_loss:0.10429
steps:16000, train_loss:0.09176, val_loss:0.10271
test done!
The time of fold 1 is: 307.16953s.
fold:2
steps:10000, train_loss:1.06643, val_loss:0.10459
steps:11000, train_loss:0.09923, val_loss:0.10097
steps:12000, train_loss:0.09442, val_loss:0.10001
steps:13000, train_loss:0.09258, val_loss:0.10003
steps:14000, train_loss:0.09423, val_loss:0.10141
steps:15000, train_loss:0.09745, val_loss:0.10505
steps:16000, train_loss:0.09251, val_loss:0.09956
test done!
The time of fold 2 is: 253.50866s.
fold:3
steps:10000, train_loss:1.07755, val_loss:0.10045
steps:11000, train_loss:0.09660, val_loss:0.09859
steps:12000, train_loss:0.09074, val_loss:0.09907
steps:13000, train_loss:0.09137, val_

In [23]:
# 后处理，提交结果

best_threshold, best_f1 = bestThreshold(train_Y, train_preds)
if use_local_test:
    print("local_test_f1:{:.5f}".format(metrics.f1_score(local_test_Y, (test_preds_local.mean(axis=1) > best_threshold))))

sub = pd.read_csv('../input/sample_submission.csv')
sub["prediction"] = (test_preds.mean(axis=1) > best_threshold).astype(int)
sub.to_csv("submission.csv", index=False)

100%|██████████| 41/41 [00:06<00:00,  5.74it/s]


best threshold is 0.3500 with F1 score: 0.6780
local_test_f1:0.69438


In [24]:
pd.DataFrame(test_preds_local).corr()

,0,1,2,3,4
0,1.000000,0.937434,0.931357,0.934701,0.939181
1,0.937434,1.000000,0.940900,0.941263,0.944434
2,0.931357,0.940900,1.000000,0.936034,0.938299
3,0.934701,0.941263,0.936034,1.000000,0.942658
4,0.939181,0.944434,0.938299,0.942658,1.000000


In [25]:
bt, bf = bestThreshold(local_test_Y, test_preds_local.mean(axis=1))

100%|██████████| 41/41 [00:00<00:00, 70.89it/s]

best threshold is 0.3600 with F1 score: 0.6945
